# Tokenizer

## Installs

In [1]:
%pip install tokenizers

  Obtaining dependency information for tokenizers from https://files.pythonhosted.org/packages/5e/8c/e32b066f3a2c924235e6a2ecad5b22c3c64b569f192975815c887b4bcfdf/tokenizers-0.15.1-cp312-none-win_amd64.whl.metadata
  Obtaining dependency information for huggingface_hub<1.0,>=0.16.4 from https://files.pythonhosted.org/packages/28/03/7d3c7153113ec59cfb31e3b8ee773f5f420a0dd7d26d40442542b96675c3/huggingface_hub-0.20.3-py3-none-any.whl.metadata
  Obtaining dependency information for filelock from https://files.pythonhosted.org/packages/81/54/84d42a0bee35edba99dee7b59a8d4970eccdd44b99fe728ed912106fc781/filelock-3.13.1-py3-none-any.whl.metadata
  Obtaining dependency information for fsspec>=2023.5.0 from https://files.pythonhosted.org/packages/70/25/fab23259a52ece5670dcb8452e1af34b89e6135ecc17cd4b54b4b479eac6/fsspec-2023.12.2-py3-none-any.whl.metadata
  Obtaining dependency information for requests from https://files.pythonhosted.org/packages/70/8e/0e2d847013cb52cd35b38c009bb167a1a26b2ce6cd696


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Imports

In [2]:
from tokenizers import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing
from tqdm.auto import tqdm
from pathlib import Path
import os

d:\alembedder2\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Directories

In [3]:
# Data directory
dir = os.getcwd()
data_folder = Path(f'{dir}/data')
files = [str(data_folder / file) for file in os.listdir(data_folder) if file.endswith('.txt')]

# Directory to save the tokenizer
output_dir = "RoBERTa_Tokenizer"

# Create the directory if it does not exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Main loop

In [4]:
for file in tqdm(files, desc="Training tokenizers"):
    # Extract filename without extension to use as a unique identifier
    filename = Path(file).stem

    # Directory to save this specific tokenizer
    output_dir = os.path.join(base_output_dir, filename)

    # Create the directory if it does not exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Initialize a tokenizer
    tokenizer = ByteLevelBPETokenizer()

    tokenizer.train(
        files=[file],
        vocab_size=52_000,
        min_frequency=2,
        special_tokens=["<s>", "<pad>", "</s>", "<unk>", "<mask>"],
    )

    # Save the tokenizer in its unique directory
    tokenizer.save_model(output_dir)

    # Post-processing: adding special tokens for RoBERTa
    tokenizer._tokenizer.post_processor = BertProcessing(
        sep=("</s>", tokenizer.token_to_id("</s>")),
        cls=("<s>", tokenizer.token_to_id("<s>")),
    )
    tokenizer.enable_truncation(max_length=512)